<div class="alert alert-block alert-success"> <center> <h1> <b> Newsfeed Dashboard v2 </b> </h1> </center></div>

<div class="alert alert-block alert-danger"> <b> BEFORE RUNNING THIS NOTEBOOK, PLEASE MAKE SURE YOU INSTALLED CORE-NLP FOR SENTIMENT ANALYSIS.</b> </div>



**To Install CoreNLP, please follow the instructions on stack overflow: [Stanford NLP for Python](https://stackoverflow.com/questions/32879532/stanford-nlp-for-python). Note that you also need Java to be installed.** 

**Once the package is installed, you only need to compute these two following lines on your terminal each time you run this code:**

$ cd stanford-corenlp-full-2018-10-05

$ java -mx5g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -timeout 10000

**If any package is not loaded, try installing it using pip install** <i><u> package name </u></i>


### Import Libraries

In [1]:
import requests
from lxml import etree
import numpy as np
import pandas as pd
import datetime as dt
import json
import shapely
import shapely.geometry
import fiona
from pycorenlp import StanfordCoreNLP
import time
from newspaper import Article
from collections import Counter

### Functions to get data from EMM

In [2]:
def datetorss(date_object):
    return date_object.strftime('%Y-%m-%d')+date_object.strftime('T%H')+'%3A'+date_object.strftime('%M')+'%3A'+date_object.strftime('%S')+'Z'

In [3]:
def get_data_cat_date(last_date,start_date,cat,countries):
    print('Running:',cat,'from:',start_date,'to:',last_date)

    original_title = list()
    links = list()
    description = list()
    pubdate = list()
    orig_lang = list()
    category = list()
    subcategory = list()
    location = list()
    trans_title = list()
    trans_descrip = list()
    entity = list()
    if countries == []:
        link = 'http://emm.newsbrief.eu/rss/rss?language=all&type=search&mode=advanced&dateto='+datetorss(last_date)+'&datefrom='+datetorss(start_date)+'&category=UNDP_'+cat+'&datefrom='+start_date.strftime('%Y-%m-%d')+'&dateto='+last_date.strftime('%Y-%m-%d') 
    else:
        link = 'http://emm.newsbrief.eu/rss/rss?language=all&type=search&mode=advanced&dateto='+datetorss(last_date)+'&datefrom='+datetorss(start_date)+'&category=UNDP_'+cat+'&datefrom='+start_date.strftime('%Y-%m-%d')+'&dateto='+last_date.strftime('%Y-%m-%d')+'&country='+countries 
    print(link)
    cont = requests.get(link)
    data_string = cont.content
    root = etree.XML(data_string)
    for element in root.iter('item'):
        original_title.append(element.find('title').text)
        links.append(element.find('link').text)
        description.append(element.find('description').text)
        pubdate.append(element.find('pubDate').text)
        orig_lang.append(element.find('{http://www.iso.org/3166}language').text)
        category.append(cat)
        u = []
        for j in element.findall('category'):
            if j.text != 'UNDP_'+cat:
                u.append(j.text)
        subcategory.append(u)
        try:
            location.append(element.find('{http://www.georss.org/georss}point').text)
        except:
            location.append(np.nan)
        try:
            trans_title.append(element.find('{http://emm.jrc.it}title').text)
            trans_descrip.append(element.find('{http://emm.jrc.it}description').text)
        except:
            trans_title.append(original_title[-1])
            trans_descrip.append(description[-1])
        m = []
        for child in element.findall('{http://emm.jrc.it}entity'):
            #print(child.attrib['name'])
            m.append(child.attrib['name'])
        entity.append(m)
    data = pd.DataFrame({'Original Title':original_title, 
                         'Link':links,
                         'Original Description':description,
                         'Publication Date':pubdate,
                         'Original Language':orig_lang,
                         'Category':category,
                         'Subcategory':subcategory,
                         'Location':location,
                         'Translated Title': trans_title,
                         'Translated Description': trans_descrip,
                         'Entity': entity})
    data['Publication Date'] = data['Publication Date'].apply(lambda date: dt.datetime.strptime(date.split(' ',1)[1],'%d %b %Y %H:%M:%S %z'))
    return data


In [4]:
def loop(cat,last_date,start_date,countries): 
    data = pd.DataFrame({'Original Title':[],
                                 'Link':[],
                                 'Original Description':[],
                                 'Publication Date':[],
                                 'Original Language':[],
                                 'Category':[],
                                 'Subcategory':[],
                                 'Location':[],
                                 'Translated Title': [],
                                 'Translated Description': [],
                                 'Entity': []})
    i = 0
    while start_date<last_date:
        data = pd.concat([data,get_data_cat_date(last_date,start_date,cat,countries)])
        print('Loop number:',i+1,'--- Data size:',len(data))
        print('----------------------')
        if len(data) < 100:
            return data
        data['Publication Date'] = data['Publication Date'].apply(lambda x: x.replace(tzinfo=None))
        data.sort_values(by = 'Publication Date',ascending = False,inplace = True)
        data.index = range(len(data))
        last_date_2 = last_date
        last_date = data['Publication Date'].min().replace(tzinfo=None)
        if last_date_2 == last_date:
            if last_date - start_date >= dt.timedelta(days = 1):
                last_date = last_date - dt.timedelta(hours = 1)
            else:
                break
        i+=1
    return data


In [5]:
def get_dataset(start_date,last_date,countries):
    categories = ['Conflict','Drought','Ecology','Genocide','HumanitarianAid','ManMadeDisasters','NaturalDisasters','PoliticalUnrest',
             'ScienceandTechnology','Security','Society','TerroristAttack','WaterConflict']
    data = pd.DataFrame({'Original Title':[],
                                 'Link':[],
                                 'Original Description':[],
                                 'Publication Date':[],
                                 'Original Language':[],
                                 'Category':[],
                                 'Subcategory':[],
                                 'Location':[],
                                 'Translated Title': [],
                                 'Translated Description': [],
                                 'Entity': []})
    for cat in categories:
        data = pd.concat([data, loop(cat,last_date,start_date,countries)])
    return data
        

### Get Full article

In [6]:
def get_article(link):
    try:
        article = Article(link)
        article.download()
        article.parse()
        return article.text
    except:
        return np.nan

## Processing the scraped dataset

### Get Article Relevance (Count of articles)

In [7]:
def get_relevant(data):
    df = data
    count = data.groupby('Original Title').count()['Link']
    count_df = pd.DataFrame({'Original Title': count.index, 'Count': count.values})
    df = df.drop_duplicates(subset = 'Original Title').merge(count_df, on = 'Original Title', how = 'left')
    return df


### Get Country

In [8]:
def distance(lat1,lon1,lat2,lon2):
    from math import sin, cos, sqrt, atan2, radians
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return(distance)

def get_country(lat, lon, borders,coord):
    try:
        closest = [(k,distance(lat,lon,coord[k][0],coord[k][1])) for k in coord.keys()]
        closest.sort(key=lambda elem: elem[1])
        # Find the associated biome
        for j in (closest):
            border_polygon = shapely.geometry.asShape(borders[j[0]]['geometry'])
            point = shapely.geometry.Point(lon, lat)
            if border_polygon.contains(point):
                country = borders[j[0]]['properties']['NAME']
                break
            else:
                country = np.nan
        return country
    except:
        return np.nan

def get_lat(x):
    try:
        return float((str(x)).split(' ')[0])
    except:
        return np.nan
    
def get_lon(x):
    try:
        return float((str(x)).split(' ')[1])
    except:
        return np.nan
        

def add_country(data,countries_borders, coord):
    data['Latitude'] = data['Location'].apply(lambda x: get_lat(x))
    data['Longitude'] = data['Location'].apply(lambda x: get_lon(x))
    data['Country'] = data.apply(lambda x: get_country(x['Latitude'],x['Longitude'], countries_borders,coord), axis = 1)
    return data

### Get Sentiment

In [9]:
def coreNLPanalysis(text,nlp):
    res = nlp.annotate(text,
                   properties={
                       'annotators': 'sentiment',
                       'outputFormat': 'json',
                       'timeout': 5000000, 'ssplit.isOneSentence': True
                   })
    try:
        return res['sentences'][0]['sentimentDistribution'] 
    except:
        return [np.nan]*5
    
def arg(x,Full_text):
    if Full_text and x['Original Language'] == 'en':
        return 'Full article'
    else:
        return 'Translated Description'
    
    
def get_sentiment(data,nlp,Full_text):
    data[['Very Neg', 'Neg', 'Neutral', 'Pos', 'Very Pos','On']] = data.apply(lambda x: coreNLPanalysis(x[arg(x,Full_text)],nlp)+[arg(x,Full_text)],
                                                                                                   axis = 1, result_type = 'expand')
    return data



### Get Emotions

In [10]:
def get_emotion_dict():
    nrc = "./External_files/NRC-emotion-lexicon-wordlevel-alphabetized-v0.92.txt"
    count=0
    emotion_dict=dict()
    with open(nrc,'r') as f:
        all_lines = list()
        for line in f:
            if count < 46:
                count+=1
                continue
            line = line.strip().split('\t')
            if int(line[2]) == 1:
                if emotion_dict.get(line[0]):
                    emotion_dict[line[0]].append(line[1])
                else:
                    emotion_dict[line[0]] = [line[1]]
    return emotion_dict

def emotion_analyzer(text,emotion_dict=get_emotion_dict()):
    #Set up the result dictionary
    emotions = {x for y in emotion_dict.values() for x in y} 
    #list comprehension - emotion for (emotion_list in emotion_dict.values() for emotion in emotion_list
    emotion_count = dict()
    for emotion in emotions:
        emotion_count[emotion] = 0

    #Analyze the text and normalize by total number of words
    total_words = len(text.split())
    for word in text.split():
        if emotion_dict.get(word):
            for emotion in emotion_dict.get(word):
                emotion_count[emotion] += 1/total_words
    # Keys: Sadness, Joy, Surprise, Anticipation, Fear, Negative, Anger, Trust, Positive, Disgust
    return list(emotion_count.values())

def get_emotion(data):
    emotion_dict = get_emotion_dict()
    data[['Sadness', 'Joy', 'Surprise', 'Anticipation', 'Fear', 'Neg', 'Anger', 'Trust', 'Positive', 'Disgust']] = data.apply(lambda x: emotion_analyzer(x['Translated Description'],emotion_dict=emotion_dict),
                                                                                                   axis = 1, result_type = 'expand')
    data.drop(['Positive', 'Neg'], axis=1, inplace = True)
    #Columns: Sadness, Joy, Surprise, Anticipation, Fear, Negative, Anger, Trust, Positive, Disgust
    return data


### Get Most common entities

In [11]:
def get_most_common(data):
    categories = ['Conflict','Drought','Ecology','Genocide','HumanitarianAid','ManMadeDisasters','NaturalDisasters','PoliticalUnrest',
             'ScienceandTechnology','Security','Society','TerroristAttack','WaterConflict']
    columns = ['Category']
    for i in range(20):
        columns.append('Most_common_'+str(i+1))
        columns.append('Count_'+str(i+1))

    data['Entity_all'] = data.apply(lambda x: x['Count']*x['Entity'],axis = 1)
    list_ = []   
    for cat in categories:
        df = data[data['Category']==cat]
        entities = sum(df['Entity_all'].as_matrix().flatten(), [])
        c = Counter(entities)
        small_list = list(sum(c.most_common(20),()))
        list_.append([cat]+small_list+(20-len(small_list)//2)*[np.nan,np.nan])
    
    list_ = np.array(list_)
    output = pd.DataFrame(list_,columns = columns)
    return output
        

In [ ]:
data = 

### Final Function

In [12]:
def final_data():
    countries = input('Enter country codes (ex: US, FR, GB/all): ')
    if countrie == 'all':
        countries = []
    else:
        countries = countries.replace(', ','%2C')
    start_date = input("Enter the begining date (yyyy/mm/dd/hh/mm/ss): ")
    while True:
        try:
            start_date = dt.datetime.strptime(start_date,'%Y/%m/%d/%H/%M/%S')
            break
        except:
            start_date = input("-\nWrong entry format.\nEnter the begining date (yyyy/mm/dd/hh/mm/ss): ")
    last_date = input("Enter the ending date (yyyy/mm/dd/hh/mm/ss): ")
    while True:
        try:
            last_date = dt.datetime.strptime(last_date,'%Y/%m/%d/%H/%M/%S')
            break
        except:
            last_date = input("-\nWrong entry format.\nEnter the begining date (yyyy/mm/dd/hh/mm/ss): ")
    Full_text = input("Perform sentiment analysis on full text? (y/n): ")
    while True:
        if Full_text != 'y' and Full_text != 'n':
            Full_text = input("-\nWrong entry format.\nPerform sentiment analysis on full text? (y/n): ")
        else:
            break
    if Full_text == 'y':
        Full_text = True
    elif Full_text == 'n':
        Full_text = False
    data = get_dataset(start_date,last_date,countries)
    print('---------------------------------------------------')
    print('Running Relevance of data')
    data = get_relevant(data)
    print('---------------------------------------------------')
    print('Running countries')
    countries_borders = fiona.open('./External_files/Country_borders/Borders.shp')
    n = 246
    coord = dict()
    for i in range(n):
        lat = float(countries_borders[i]['properties']['Info de _1'])
        lon = float(countries_borders[i]['properties']['Info de gÃ'])
        coord[i] = (lat,lon)
    data = add_country(data,countries_borders,coord)
    if Full_text:
        print('---------------------------------------------------')
        print('Running full text')
        data['Full Article'] = data['Link'].apply(lambda x: get_article(x))
        
    print('---------------------------------------------------')
    
    while True:
        try:
            nlp = StanfordCoreNLP('http://localhost:9000')
            print('Running Sentiment')
            print("Success")
            data = get_sentiment(data,nlp,Full_text)
            break
        except:
            print('Running Sentiment')
            print('Unable. Please connect to CoreNLP server')
            time.sleep(20)
    print('---------------------------------------------------')
    print('Running Emotions')
        
    data = get_emotion(data)
    data2 = get_most_common(data)
    return data, data2

In [16]:
data, data2 = final_data()

Enter the begining date (yyyy/mm/dd/hh/mm/ss): 2019/04/01/00/00/00
Enter the ending date (yyyy/mm/dd/hh/mm/ss): 2019/04/01/23/59/00
Perform sentiment analysis on full text? (y/n): n
Running: Conflict from: 2019-04-01 00:00:00 to: 2019-04-01 23:59:00
http://emm.newsbrief.eu/rss/rss?language=all&type=search&mode=advanced&dateto=2019-04-01T23%3A59%3A00Z&datefrom=2019-04-01T00%3A00%3A00Z&category=UNDP_Conflict&datefrom=2019-04-01&dateto=2019-04-01
Loop number: 1 --- Data size: 100
----------------------
Running: Conflict from: 2019-04-01 00:00:00 to: 2019-04-01 22:35:00
http://emm.newsbrief.eu/rss/rss?language=all&type=search&mode=advanced&dateto=2019-04-01T22%3A35%3A00Z&datefrom=2019-04-01T00%3A00%3A00Z&category=UNDP_Conflict&datefrom=2019-04-01&dateto=2019-04-01
Loop number: 2 --- Data size: 200
----------------------
Running: Conflict from: 2019-04-01 00:00:00 to: 2019-04-01 21:46:00
http://emm.newsbrief.eu/rss/rss?language=all&type=search&mode=advanced&dateto=2019-04-01T21%3A46%3A00Z&d

Loop number: 1 --- Data size: 100
----------------------
Running: HumanitarianAid from: 2019-04-01 00:00:00 to: 2019-04-01 17:37:00
http://emm.newsbrief.eu/rss/rss?language=all&type=search&mode=advanced&dateto=2019-04-01T17%3A37%3A00Z&datefrom=2019-04-01T00%3A00%3A00Z&category=UNDP_HumanitarianAid&datefrom=2019-04-01&dateto=2019-04-01
Loop number: 2 --- Data size: 200
----------------------
Running: HumanitarianAid from: 2019-04-01 00:00:00 to: 2019-04-01 15:17:00
http://emm.newsbrief.eu/rss/rss?language=all&type=search&mode=advanced&dateto=2019-04-01T15%3A17%3A00Z&datefrom=2019-04-01T00%3A00%3A00Z&category=UNDP_HumanitarianAid&datefrom=2019-04-01&dateto=2019-04-01
Loop number: 3 --- Data size: 300
----------------------
Running: HumanitarianAid from: 2019-04-01 00:00:00 to: 2019-04-01 12:35:00
http://emm.newsbrief.eu/rss/rss?language=all&type=search&mode=advanced&dateto=2019-04-01T12%3A35%3A00Z&datefrom=2019-04-01T00%3A00%3A00Z&category=UNDP_HumanitarianAid&datefrom=2019-04-01&dateto=

Loop number: 2 --- Data size: 200
----------------------
Running: NaturalDisasters from: 2019-04-01 00:00:00 to: 2019-04-01 21:11:00
http://emm.newsbrief.eu/rss/rss?language=all&type=search&mode=advanced&dateto=2019-04-01T21%3A11%3A00Z&datefrom=2019-04-01T00%3A00%3A00Z&category=UNDP_NaturalDisasters&datefrom=2019-04-01&dateto=2019-04-01
Loop number: 3 --- Data size: 300
----------------------
Running: NaturalDisasters from: 2019-04-01 00:00:00 to: 2019-04-01 20:23:00
http://emm.newsbrief.eu/rss/rss?language=all&type=search&mode=advanced&dateto=2019-04-01T20%3A23%3A00Z&datefrom=2019-04-01T00%3A00%3A00Z&category=UNDP_NaturalDisasters&datefrom=2019-04-01&dateto=2019-04-01
Loop number: 4 --- Data size: 400
----------------------
Running: NaturalDisasters from: 2019-04-01 00:00:00 to: 2019-04-01 19:42:00
http://emm.newsbrief.eu/rss/rss?language=all&type=search&mode=advanced&dateto=2019-04-01T19%3A42%3A00Z&datefrom=2019-04-01T00%3A00%3A00Z&category=UNDP_NaturalDisasters&datefrom=2019-04-01&d

Loop number: 4 --- Data size: 400
----------------------
Running: Security from: 2019-04-01 00:00:00 to: 2019-04-01 20:03:00
http://emm.newsbrief.eu/rss/rss?language=all&type=search&mode=advanced&dateto=2019-04-01T20%3A03%3A00Z&datefrom=2019-04-01T00%3A00%3A00Z&category=UNDP_Security&datefrom=2019-04-01&dateto=2019-04-01
Loop number: 5 --- Data size: 500
----------------------
Running: Security from: 2019-04-01 00:00:00 to: 2019-04-01 19:46:00
http://emm.newsbrief.eu/rss/rss?language=all&type=search&mode=advanced&dateto=2019-04-01T19%3A46%3A00Z&datefrom=2019-04-01T00%3A00%3A00Z&category=UNDP_Security&datefrom=2019-04-01&dateto=2019-04-01
Loop number: 6 --- Data size: 600
----------------------
Running: Security from: 2019-04-01 00:00:00 to: 2019-04-01 19:44:00
http://emm.newsbrief.eu/rss/rss?language=all&type=search&mode=advanced&dateto=2019-04-01T19%3A44%3A00Z&datefrom=2019-04-01T00%3A00%3A00Z&category=UNDP_Security&datefrom=2019-04-01&dateto=2019-04-01
Loop number: 7 --- Data size: 7

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]


In [17]:
data2

,Category,Most_common_1,Count_1,Most_common_2,Count_2,Most_common_3,Count_3,Most_common_4,Count_4,Most_common_5,...,Most_common_16,Count_16,Most_common_17,Count_17,Most_common_18,Count_18,Most_common_19,Count_19,Most_common_20,Count_20
0,Conflict,World Health Organization,218,United Nations,184,EU,94,The Lancet,90,Christian Lindmeier,...,İstanbul Buyukşehir Belediyesi,46,Muhammad Al Sabah Al Salim Al Sabah,44,Second World War,39,Mark Lowcock,38,Peter Maurer,38
1,Drought,20 Minutes,19,El Nino,16,United Nations,15,Val Seriana,14,World Vision,...,Second World War,6,Dabo Guan,5,University of East Anglia,5,Nuevo Diario,5,German government,5
2,Ecology,EU,77,United Nations,42,IUCN,20,Agriculture Ministry,17,Leif Miller,...,بن طلال,10,Environmental Protection Agency,10,UNESCO,10,Masai Mara,9,European Commission,9
3,Genocide,International Criminal Court,24,Agathe Uwilingiyimana,20,Bernard Ntuyahaga,20,Paul Kagame,16,Jean Baptiste Ouedraogo,...,Aung San Suu Kyi,5,Washington Post,5,American Service,4,Chile Eboe Osuji,4,Member States,4
4,HumanitarianAid,Donald Trump,160,Kirstjen Nielsen,130,Kellyanne Conway,130,Washington Post,129,Mick Mulvaney,...,Filipe Nyusi,27,Kay Bailey,26,Angela Merkel,26,Christlich Soziale Union in Bayern,26,Kay Bailey Hutchison,26
5,ManMadeDisasters,John Flesher,137,Gretchen Whitmer,131,Dana Nessel,131,U S Coast Guard,131,National Wildlife,...,محمد بركة,58,Mohamed Lahouaiej-Bouhlel,50,Hamdi Zagar,50,Associated Press,49,Kia Soul,49
6,NaturalDisasters,United Nations,76,Associated Press,62,World Health Organization,46,Giuseppe Fiorini Morosini,40,Emmanuel Macron,...,Hurricane Maria,21,Lola Castro,20,Francesco Massaro,20,Sante Guido,20,San Severino,20
7,PoliticalUnrest,EU,100,European Commission,56,Zoltan Kovacs,48,European Parliament,46,Emmanuel Macron,...,Theresa May,20,Михаила Русого,18,Donald Trump,17,Abdelaziz Bouteflika,16,Prof Dr,16
8,ScienceandTechnology,Nasdaq Composite,68,EU,45,Google,45,Securities and Exchange Commission,33,Pfizer Inc,...,Bristol Myers Squibb,16,Howard Hughes,16,Rockefeller University,16,Leslie Vosshall,16,Ben Matthews,16
9,Security,UN Security Council,138,Air Force,56,United Nations,46,FPOE,46,AK Parti,...,Al Noor,27,Jaish-e-Mohammed,27,Saadet Partisi,25,Organization for Security and Co operation in ...,25,State Department,25
